In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
from torch.utils.data import Subset, DataLoader
import torchvision
from torch import optim
from tqdm import tqdm
from torch.utils.data import DataLoader
from torchvision import datasets
import torchvision.models as models

import math

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
eng_characters = [chr(i) for i in range(ord('a'), ord('z')+1)]  # English lowercase letters
eng_characters += [chr(i) for i in range(ord('A'), ord('Z')+1)]  # English uppercase letters
eng_characters += [chr(i) for i in range(ord('0'), ord('9')+1)]  # Digits
eng_characters += ['?','<','>']
# Add other characters as needed, such as punctuation marks, special symbols, etc.

print(eng_characters)
# Define the range of Unicode code points for Telugu characters
TELUGU_START = 0x0C00
TELUGU_END = 0x0C7F

# Generate all Telugu characters
telugu_characters = [chr(code_point) for code_point in range(TELUGU_START, TELUGU_END + 1)]
telugu_characters += ['?','<','>']

print(telugu_characters)



['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '?', '<', '>']
['ఀ', 'ఁ', 'ం', 'ః', 'ఄ', 'అ', 'ఆ', 'ఇ', 'ఈ', 'ఉ', 'ఊ', 'ఋ', 'ఌ', '\u0c0d', 'ఎ', 'ఏ', 'ఐ', '\u0c11', 'ఒ', 'ఓ', 'ఔ', 'క', 'ఖ', 'గ', 'ఘ', 'ఙ', 'చ', 'ఛ', 'జ', 'ఝ', 'ఞ', 'ట', 'ఠ', 'డ', 'ఢ', 'ణ', 'త', 'థ', 'ద', 'ధ', 'న', '\u0c29', 'ప', 'ఫ', 'బ', 'భ', 'మ', 'య', 'ర', 'ఱ', 'ల', 'ళ', 'ఴ', 'వ', 'శ', 'ష', 'స', 'హ', '\u0c3a', '\u0c3b', '\u0c3c', 'ఽ', 'ా', 'ి', 'ీ', 'ు', 'ూ', 'ృ', 'ౄ', '\u0c45', 'ె', 'ే', 'ై', '\u0c49', 'ొ', 'ో', 'ౌ', '్', '\u0c4e', '\u0c4f', '\u0c50', '\u0c51', '\u0c52', '\u0c53', '\u0c54', 'ౕ', 'ౖ', '\u0c57', 'ౘ', 'ౙ', 'ౚ', '\u0c5b', '\u0c5c', '\u0c5d', '\u0c5e', '\u0c5f', 'ౠ', 'ౡ', 'ౢ', 'ౣ', '\u0c64', '\u0c65', '౦', '౧', '౨', '౩', '౪', '౫', '౬', '౭', '౮', '౯'

In [3]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_train.csv'
valid_file = '/kaggle/input/aksharantar2/aksharantar_sampled/tel/tel_valid.csv'
# Load train and validation datasets

# Load the datasets
train_dataset = pd.read_csv(train_file, header=None)
valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
source_data = train_dataset.iloc[:, 0].values
target_data = train_dataset.iloc[:, 1].values


In [4]:
source_data = ['<' + s + '>' for s in source_data]
target_data = ['<' + t + '>' for t in target_data]

In [5]:
loc = 1
e,t = train_dataset.iloc[loc]
char_to_index_eng = {char: i for i, char in enumerate(eng_characters)}
index_to_char_eng = {i: char for char, i in char_to_index_eng.items()}
def char_to_tensor_eng(word, char_to_index_eng):
    return torch.tensor([char_to_index_eng[char] for char in word], dtype=torch.long)

print(e)
print(char_to_tensor_eng(e,char_to_index_eng))

char_to_index_tel = {char: i for i, char in enumerate(telugu_characters)}
index_to_char_tel = {i: char for char, i in char_to_index_tel.items()}
def char_to_tensor_tel(word, char_to_index_tel):
    return torch.tensor([char_to_index_tel[char] for char in word], dtype=torch.long)

print(t)
t1 = char_to_tensor_tel(t,char_to_index_tel)
print(t1)

characters = [telugu_characters[char_int.item()] for char_int in t1]
string = ''.join(characters)
# Print the characters
print("Characters:", string)
# for batch_idx, (data, target) in enumerate(train_loader):
#     print(f"Batch {batch_idx}:")
#     print("Data shape:", data.shape)
#     print("Target shape:", target.shape)
#     # You can print more details about the data and target if needed
#     break  # This will break after printing the first batch

vastadira
tensor([21,  0, 18, 19,  0,  3,  8, 17,  0])
వస్తాదిరా
tensor([53, 56, 77, 36, 62, 38, 63, 48, 62])
Characters: వస్తాదిరా


In [6]:

source = [char_to_tensor_eng(s, char_to_index_eng) for s in source_data]
target = [char_to_tensor_tel(s, char_to_index_tel) for s in target_data]

In [7]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import TensorDataset
# Pad the tensors to the same length

source_padded = pad_sequence(source, batch_first=True, padding_value=62)
target_padded = pad_sequence(target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(source_padded.size(1), target_padded.size(1))

# Pad source and target to the same length
source_padded = torch.nn.functional.pad(source_padded, (0, max_length - source_padded.size(1)), value=62)
target_padded = torch.nn.functional.pad(target_padded, (0, max_length - target_padded.size(1)), value=128)

# Add 62 at the beginning and 63 at the end of each item in source_padded
# source_padded = torch.cat((torch.full((source_padded.shape[0], 1), 63, dtype=torch.long), 
#                            source_padded, torch.full((source_padded.shape[0], 1), 64, dtype=torch.long)), dim=1)
# target_padded = torch.cat((torch.full((target_padded.shape[0], 1), 129, dtype=torch.long), 
#                            target_padded, torch.full((target_padded.shape[0], 1), 130, dtype=torch.long)), dim=1)

In [8]:
target_padded[0]

tensor([129,  53,  48,  77,  23,  62,  50,  53,  62,  48,  63,  40,  71, 130,
        128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128, 128,
        128, 128])

In [9]:
def t2p(s,t):
    characters = [eng_characters[char_int.item()] for char_int in s]
    string1 = ''.join(characters)
#     print(string)
    characters = [telugu_characters[char_int.item()] for char_int in t]
    string2 = ''.join(characters)
    print(f"{string1} {len(string1)} : {string2} {len(string2)} :")
t2p(source_padded[0],target_padded[0])

<vargaalavaarine>????????????? 30 : <వర్గాలవారినే>???????????????? 30 :


In [10]:
# Create a TensorDataset
dataset = TensorDataset(source_padded, target_padded)

# Create a DataLoader
batch_size = 64
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Example usage of the DataLoader
cnt=1
for batchid,(s,t) in enumerate(tqdm(train_dataloader)):
    #source_batch, target_batch = batch
    # Your training code here
    print(s[0])
    for item_s, item_t in zip(s, t):
#     print(f"{batchid}:")
        t2p(item_s,item_t)
#     print(s)
#     cnt+=1
#     if(cnt>2):break
    if(batchid>0): break


  0%|          | 1/800 [00:00<00:27, 28.62it/s]

tensor([63, 10,  0, 19, 19,  0, 11,  0, 13, 20, 10, 20, 13, 19, 14, 13,  3,  8,
        64, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62, 62])
<kattalanukuntondi>??????????? 30 : <కట్టాలనుకుంటోంది>???????????? 30 :
<arkula>?????????????????????? 30 : <అర్కుల>?????????????????????? 30 :
<nalga>??????????????????????? 30 : <నల్గ>???????????????????????? 30 :
<beetrootnu>?????????????????? 30 : <బీట్రూట్ను>?????????????????? 30 :
<seppadu>????????????????????? 30 : <సెప్పడు>????????????????????? 30 :
<okkarojunu>?????????????????? 30 : <ఒక్కరోజును>?????????????????? 30 :
<vyavasaayadaarlu>???????????? 30 : <వ్యవసాయదార్లు>??????????????? 30 :
<antagoppa>??????????????????? 30 : <అంతగొప్ప>???????????????????? 30 :
<yarapati>???????????????????? 30 : <యరపతి>??????????????????????? 30 :
<dwipatra>???????????????????? 30 : <ద్విపాత్ర>??????????????????? 30 :
<godavaladukuntu>????????????? 30 : <గొడవలాడుకుంటూ>??????????????? 30 :
<kanganapai>?????????????????? 30 : <కంగనపై>?????????????????????? 30 :

In [11]:
#valid_dataset = pd.read_csv(valid_file, header=None)

# Extract the source and target columns
val_source_data = valid_dataset.iloc[:, 0].values
val_target_data = valid_dataset.iloc[:, 1].values

val_source_data = ['<' + s + '>' for s in val_source_data]
val_target_data = ['<' + t + '>' for t in val_target_data]

val_source = [char_to_tensor_eng(s, char_to_index_eng) for s in val_source_data]
val_target = [char_to_tensor_tel(s, char_to_index_tel) for s in val_target_data]

val_source_padded = pad_sequence(val_source, batch_first=True, padding_value=62)
val_target_padded = pad_sequence(val_target, batch_first=True, padding_value=128)

# Find the maximum length
max_length = max(val_source_padded.size(1), val_target_padded.size(1))

# Pad source and target to the same length
val_source_padded = torch.nn.functional.pad(val_source_padded, (0, max_length - val_source_padded.size(1)), value=62)
val_target_padded = torch.nn.functional.pad(val_target_padded, (0, max_length - val_target_padded.size(1)), value=128)
# Create a TensorDataset
val_dataset = TensorDataset(val_source_padded, val_target_padded)


valid_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [25]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.cell_type = cell_type
        self.embeddin_size = embedding_size
        # Character embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, batch_first=True)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x)
        # embedded shape: (seq_length:max_length, N(batch size:32), embedding_size:100)
        output, hidden = self.rnn(embedded, hidden)
        return hidden
    
    def getInitialState(self):
        return torch.zeros(self.num_layers,self.batch_size,self.hidden_size).to(device)


class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, 
                 output_size, num_layers, cell_type):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.cell_type = cell_type
        self.embeddin_size = embedding_size
        
        # Devanagari character embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
        
        # Decoder RNN
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=2)
    
    def forward(self, x, hidden):
        # shape of x: (N) but required (1, N)
#         x = x.unsqueeze(0)
        
        embedded = self.embedding(x)
        # embedded shape: (1, N, embedding_size) #n * 1 * embd
        output, hidden = self.rnn(embedded, hidden)  
        # shape of outputs: (N*1*131)
        predictions = self.softmax(self.fc(output))
        # shape of predictionos:  (1, N, length of tel_char)
#         predictions = predictions.squeeze(0)

        
        return predictions, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, cell_type):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.cell_type = cell_type
    
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = source.shape[0]
        target_len = target.shape[1]
        target_letter_size = 131
#         print(source.shape)
#         print(target.shape)
        
        outputs = torch.zeros(target_len, batch_size, target_letter_size).to(device)
        
        # Encoder forward pass
        if(self.cell_type == "LSTM"):
            hidden = (self.encoder.getInitialState(), self.encoder.getInitialState())
        else:
            hidden = self.encoder.getInitialState()
#         hidden.to(device)
        hidden = self.encoder(source, hidden)
        
#         print(hidden.shape)
        x = target[:,0].view(batch_size, 1)
#         print( "target :",x.shape)
#         print(x)
        
#         # Decoder initial hidden state
#         decoder_hidden = encoder_hidden        
#         # Decoder input (start token)
#         decoder_input = torch.tensor([[SOS_token]] * batch_size)      
        # Decoder forward pass
    
        for t in range(0, target_len):
            output, hidden = self.decoder(x, hidden)
            output = output[:, -1, :]
            outputs[t] = output
            best_guess = output.argmax(1)
            best_guess = best_guess.view(batch_size, 1) 
            x = target[:,t].view(batch_size,1) if np.random.rand() < teacher_forcing_ratio else best_guess
        
        return outputs


In [13]:
input_size = len(char_to_index_eng)
print(input_size)
char_to_index_eng['?']

65


62

In [20]:
# Define the model architecture
input_size = len(char_to_index_eng)
output_size = len(char_to_index_tel)
# hidden_size = 256
# embedding_size = 256
# num_layers = 1
# cell_type = 'LSTM'
# LEARNING_RATE = 0.0001
# NUM_EPOCHS = 10

def train(hidden_size, embedding_size, 
          num_layers, cell_type, LEARNING_RATE, NUM_EPOCHS):

    encoder_net = Encoder(input_size, embedding_size, hidden_size, num_layers, cell_type, batch_size).to(device)
    decoder_net = Decoder(output_size, embedding_size, hidden_size, 
                     output_size, num_layers, cell_type).to(device)
    model = Seq2Seq(encoder_net, decoder_net, cell_type).to(device)

    criterion = nn.NLLLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    step = 0
    # Train the model
    for epoch in range(NUM_EPOCHS):
        model.train()
        total_loss = 0
        accuracy = 0
        for batchid,(inp_data,target) in enumerate(tqdm(train_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target)
            indices = output.argmax(2)

            if(batchid==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])

            optimizer.zero_grad()
            loss = 0
            total_correct = 0
            for i in range(output.shape[0]):
                loss += criterion(output[i], target[:,i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            total_loss += loss.item()/ len(inp_data[0])
            accuracy += total_correct/batch_size
            # Backpropagation
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),max_norm=1)
            optimizer.step()
            step+=1

        train_loss = total_loss / len(train_dataloader)
        train_accuracy = accuracy
        validation_loss, validation_accuracy = calculate_validation_loss(model, criterion, valid_dataloader)
        
        print(f"Epoch {epoch+1}, Loss: {total_loss / len(train_dataloader)}")
        print("train_acc", train_accuracy)
        print("train_loss", train_loss)
        print("valid_acc", validation_accuracy)
        print("valid_loss", validation_loss)
        
        wandb.log({'train_accuracy':train_accuracy})
        wandb.log({'train_loss':train_loss})
        wandb.log({'val_accuracy':validation_accuracy})
        wandb.log({'val_loss':validation_loss})


In [21]:
def calculate_validation_loss(model, criterion, valid_dataloader):
    model.eval()  # Set the model to evaluation mode
    total_val_loss = 0
    accuracy = 0
    step = 0
    with torch.no_grad():
        for batchid,(inp_data,target) in enumerate(tqdm(valid_dataloader)):
            inp_data = inp_data.to(device)
            target = target.to(device)

            output = model(inp_data, target)
            indices = output.argmax(2)
            output_dim = output.shape[-1]
            if(batchid==0):
                t2p(inp_data[0],target[0])
                t2p(inp_data[0],indices[:,0])
                print((indices[:,0] == target[0]).sum().item())

            # Flatten the output and target tensors to compute the loss
            val_loss = 0
            total_correct = 0
            for i in range(output.shape[0]):
                val_loss += criterion(output[i], target[:, i])
                total_correct += ((indices[:, i] == target[i]).sum().item() == len(target[i]))
            step +=1
            total_val_loss += val_loss.item() / len(inp_data[0])
            accuracy += total_correct/batch_size

    avg_val_loss = total_val_loss / len(valid_dataloader)
    accuracy = accuracy #/ len(valid_dataloader)
    return avg_val_loss, accuracy


In [16]:
# train(hidden_size, embedding_size, 
#           num_layers, cell_type, LEARNING_RATE, NUM_EPOCHS)

In [17]:
!pip install wandb
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [27]:
def main_fun():
    wandb.init(project ='Assignment3')
    params = wandb.config
    with wandb.init(project = 'Assignment3', name='epochs_'+str(params.num_epochs) 
                    +'layers_'+str(params.num_layers) + 'embedding_'+str(params.embedding_size)
                    +'hidden_size_'+str(params.hidden_size) + 'cell_type_'+str(params.cell_type)
                    +'lear_rate_'+str(params.learning_rate)) as run:
        train(params.hidden_size,params.embedding_size,
              params.num_layers, params.cell_type, params.learning_rate, params.num_epochs)

sweep_params = {
    'method' : 'bayes',
    'name'   : 'cs23m005',
    'metric' : {
        'goal' : 'maximize',
        'name' : 'val_accuracy',
    },
    'parameters' : {
            'num_epochs':{'values':[20]},
            'learning_rate' :{'values':[1e-4]},
            'num_layers' :{'values':[2]},
            'embedding_size' :{'values':[256]},
            'hidden_size' :{'values':[256]},
            'cell_type' :{'values':['LSTM']},
            
    }
}
sweepId = wandb.sweep(sweep_params,project = 'Assignment3')
wandb.agent(sweepId,function =main_fun,count = 1)
wandb.finish()

Create sweep with ID: ifvksbxw
Sweep URL: https://wandb.ai/cs23m005/Assignment3/sweeps/ifvksbxw


wandb: Agent Starting Run: ggcotcuw with config:
wandb: 	cell_type: LSTM
wandb: 	embedding_size: 256
wandb: 	hidden_size: 256
wandb: 	learning_rate: 0.0001
wandb: 	num_epochs: 20
wandb: 	num_layers: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


wandb: WARNING Ignored wandb.init() arg project when running a sweep.


  0%|          | 2/800 [00:00<00:59, 13.32it/s]

<prameeta>???????????????????? 30 : <ప్రమీత>?????????????????????? 30 :
<prameeta>???????????????????? 30 : ౛ఄ౛ీీీఇఇఇఝఝఝఝ౧౧ఝఝఝ౧ఝఝ౧౧౧౧౧౧ఝ౧౧ 30 :


...

 16%|█▌        | 10/64 [00:00<00:01, 48.01it/s]

<bheeshmudini>???????????????? 30 : <భీష్ముడిని>?????????????????? 30 :
<bheeshmudini>???????????????? 30 : <భీష్ముడిని>?????????????????? 30 :
30


100%|██████████| 64/64 [00:01<00:00, 48.67it/s]


Epoch 20, Loss: 0.1782167966763179
train_acc 157.3125
train_loss 0.1782167966763179
valid_acc 11.671875
valid_loss 0.1726856838911772


train_accuracy,▁▁▁▁▁▁▂▂▃▄▄▅▅▆▆▇▇▇██
train_loss,█▆▅▅▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁▁
val_accuracy,▁▁▁▁▂▂▃▄▄▅▆▆▇▆▇▇████
val_loss,█▇▆▅▄▃▃▃▂▂▂▂▁▂▁▁▁▁▁▁
train_accuracy,157.3125
train_loss,0.17822
val_accuracy,11.67188
val_loss,0.17269
